# Debugging live strategy

In this notebook, we show how to debug live trading strategy instance using Python and Jupyter notebook.

- Downloading the trades from the live instance
- Reading data to `State` object
- Inspecting `State` in Python

## Download live data

- Each strategy Docker instances offers a webhook that allows access the data of this strategy, include `State` object
  that is the flat file state of the all the strategy current and past decision making
- Note that `State` class and any children classes in the state tree may have changes  between
  versions and decoding might need you to use a specific version of `trade-executor`


In [1]:
from tradeexecutor.monkeypatch.dataclasses_json import patch_dataclasses_json
from tradeexecutor.state.state import State
import requests

# Currently needed because unpatched dataclasses_json package issues
patch_dataclasses_json()

# Public internet endpoint as exposed by the trade executor Docker
webbhook_url = "https://pancake-eth-usd-sma.tradingstrategy.ai"

state_api = f"{webbhook_url}/state"

resp = requests.get(state_api)

state_blob = resp.content

print(f"Downloaded {len(state_blob):,} bytes state data")

state = State.from_json(state_blob)

print(f"trade-executor was launched at: {state.created_at}, we have {len(list(state.portfolio.get_all_trades()))} trades")

Downloaded 3,727,546 bytes state data
trade-executor was launched at: 2022-12-01 21:17:08, we have 250 trades


## Examining data

You can check what individual persistent objects are made of.

In this example, we have one blockchain trade whose transactions were broadcasted to the blockchain,
but for which we never got transaction receipts back to the blockchain.

In [3]:
# Inspect a single position
position = next(iter(state.portfolio.closed_positions.values()))
print(position)

print("Value: ", position.get_value())
print("Quantity: ", position.get_quantity())
print("Successful trades:")
for t in position.get_successful_trades():
    print(f"     {t}")

print("Failed trades:")
for t in position.get_failed_trades():
    print(f"     {t}")

print("Any trades:")
for trade_id, t in position.trades.items():
    print(f"     {t}")

<Closed position #1 <Pair ETH-USDC at 0xea26b78255df2bbc31c1ebf60010d78670185bd0 (0.0000% fee) on exchange 0xca143ce32fe78f1f7019d7d551a6402fc5350c73> $349.14808504607964>
Value:  0.0
Quantity:  0
Successful trades:
   <Buy #1 0.2702005902207243036333094103 ETH at 1292.1810598595061, success>
   <Sell #2 0.270336092336264209 ETH at 1281.6539976918418, success>
Failed trades:
Any trades:
   <Buy #1 0.2702005902207243036333094103 ETH at 1292.1810598595061, success>
   <Sell #2 0.270336092336264209 ETH at 1281.6539976918418, success>
